In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import dask.dataframe as dd
import numpy as np
from dask.distributed import Client
import dask.multiprocessing
#


In [2]:
import umap
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.express as px
import chart_studio.plotly as py
#

In [3]:
client = Client(n_workers=12, threads_per_worker = 2, memory_limit='56GB')
#

/home/benwright/.local/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43755 instead
  warnings.warn(


In [4]:
conn = 'sqlite:///crawl.db'
#

In [5]:
df = dd.read_sql_table('pages',conn,'id')
#

In [6]:
df.head(npartitions=1)

,url,content,added,updated
id,,,,
1,https://en.wikipedia.org/wiki/List_of_jazz_albums,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 14:38:37.118265,2024-10-12 14:38:37.303798
2,https://en.wikipedia.org/wiki/Template:Jazz_th...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 14:38:43.218992,2024-10-12 14:38:49.186496
3,https://en.wikipedia.org/wiki/Jimmy_Smith_(mus...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 14:38:43.226498,2024-10-12 14:38:49.267377
4,https://en.wikipedia.org/wiki/The_Rajah_(album),"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 14:38:43.238258,2024-10-12 14:38:49.345036
5,https://en.wikipedia.org/wiki/Blues_and_the_So...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 14:38:43.246004,2024-10-12 14:38:49.386667


In [7]:
len(df)

119833

In [8]:
def is_soup_album_page(soup):

    al = 'Track listing'

    is_album = (soup.find(string=al) == al)

    return is_album

def is_soup_song_page(soup):
    return (soup.title.text.find('(song)') > -1)

def get_title(soup):
    return soup.title.text.replace(' - Wikipedia', '').replace('- Wikipedia','')

def get_album_artist(album_soup):

    try:
        artist = album_soup.find(string=' by ').parent.find(class_='contributor').find('a')
        return artist.text, artist.get('href')
    except:
        return None, None
    
# a quick check for if a beautiful soup is for an artist article
def is_soup_artist_page(soup):

    score = 0

    should_have_strings = ('Born','Genres','Background information','Discography','Instrument(s)')
    s1 = 1.0/len(should_have_strings)

    for search_string in should_have_strings:
        if(soup.find(string=search_string)):
            score += s1

    is_artist_page = (score >= .8)

    return is_artist_page

# a quick check for if a beautiful soup is for a genre article
def is_soup_genre_page(soup):

    score = 0
    should_have_strings = ('Stylistic origins','Cultural origins','Derivative forms')
    s1 = 1.0/len(should_have_strings)

    for search_string in should_have_strings:
        if(soup.find(string=search_string)):
            score += s1

    is_genre_page = (score >= .66)

    return is_genre_page

# gets the type of wikipedia page. wikipedia specific, slow, and not guaranteed to be accurate, so off by default
def get_pagetype(soup = None, html = None):

    ret_val = ''

    if((html==None or html =='') and soup==None):
        return ret_val
    
    elif(soup == None):
        soup = BeautifulSoup(html,'html.parser')

    if(is_soup_album_page(soup)):
        return 'album'
    elif(is_soup_artist_page(soup)):
        return 'artist'
    elif(is_soup_genre_page(soup)):
        return 'genre'
    elif(is_soup_song_page(soup)):
        return 'song'
    
    return ret_val
#

In [9]:
def get_pagetype_for_df_wrapper(row):
    html = row['content']
    return get_pagetype(html=html)

meta_df = pd.DataFrame(columns=['pagetype'],dtype=str)
#

In [10]:
df['pagetype'] = df.apply(get_pagetype_for_df_wrapper,axis=1,meta=meta_df).compute()
#

2024-10-12 18:48:33,602 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2024-10-12 18:48:35,073 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2024-10-12 18:48:36,207 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-10-12 18:48:37,360 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2024-10-12 18:48:37,429 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-10-12 18:48:38,754 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2024-10-12 18:48:38,971 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-10-12 18:48:40,021 - distributed.utils_perf - WARNING - full garbage collections took

In [11]:
df2 = df[df['pagetype']!='']
#


In [12]:
def get_title_wrapper(row):
    html = row['content']
    return get_title(BeautifulSoup(html,'html.parser'))

meta_df = pd.DataFrame(columns=['title'],dtype=str)


In [13]:
df2['title'] = df2.apply(get_title_wrapper,axis=1,meta=meta_df).compute()


2024-10-12 19:09:36,718 - distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)
2024-10-12 19:09:37,894 - distributed.utils_perf - WARNING - full garbage collections took 29% CPU time recently (threshold: 10%)
2024-10-12 19:09:38,680 - distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
2024-10-12 19:09:41,203 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
2024-10-12 19:09:42,573 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
2024-10-12 19:09:43,899 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
2024-10-12 19:09:45,336 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
2024-10-12 19:09:46,813 - distributed.utils_perf - WARNING - full garbage collections took

In [14]:
df2.head(npartitions=5)

2024-10-12 19:12:18,269 - distributed.utils_perf - WARNING - full garbage collections took 42% CPU time recently (threshold: 10%)


,url,content,added,updated,pagetype,title
id,,,,,,
3,https://en.wikipedia.org/wiki/Jimmy_Smith_(mus...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 14:38:43.226498,2024-10-12 14:38:49.267377,artist,Jimmy Smith (musician)
4,https://en.wikipedia.org/wiki/The_Rajah_(album),"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 14:38:43.238258,2024-10-12 14:38:49.345036,album,The Rajah (album)
5,https://en.wikipedia.org/wiki/Blues_and_the_So...,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 14:38:43.246004,2024-10-12 14:38:49.386667,album,Blues and the Soulful Truth
6,https://en.wikipedia.org/wiki/Tommy_Flanagan,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 14:38:43.253413,2024-10-12 14:38:49.521566,artist,Tommy Flanagan
7,https://en.wikipedia.org/wiki/Lee_Morgan_Vol._3,"<!DOCTYPE html> <html class=""client-nojs vecto...",2024-10-12 14:38:43.261858,2024-10-12 14:38:49.591953,album,"Lee Morgan, Vol. 3"


In [15]:
song_df = df2[df2['pagetype']=='song']
genre_df = df2[df2['pagetype']=='genre']
album_df = df2[df2['pagetype']=='album']
artist_df = df2[df2['pagetype']=='artist']
#

In [16]:
def get_album_artist_wrapper(row):
    html = row['content']
    return get_album_artist(BeautifulSoup(html,'html.parser'))[0]

meta_df = pd.DataFrame(columns=['album_artist'],dtype=str)


In [17]:
album_df['album_artist'] = album_df.apply(get_album_artist_wrapper,axis=1,meta=meta_df).compute()


2024-10-12 19:12:27,959 - distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
2024-10-12 19:12:29,521 - distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
2024-10-12 19:12:30,590 - distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
2024-10-12 19:12:31,891 - distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
2024-10-12 19:12:33,079 - distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
2024-10-12 19:12:34,193 - distributed.utils_perf - WARNING - full garbage collections took 38% CPU time recently (threshold: 10%)
2024-10-12 19:12:34,273 - distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
2024-10-12 19:12:34,403 - distributed.utils_perf - WARNING - full garbage collections took

In [18]:
def get_label(row):
    album = row['title'].replace(' - Wikipedia', '').replace('- Wikipedia', '')
    artist = row['album_artist']
    return f"{album} by {artist}"
meta_df = pd.DataFrame(columns=['label'],dtype=str)
album_df['label'] = album_df.apply(get_label,axis=1,meta=meta_df).compute()

2024-10-12 19:14:05,688 - distributed.utils_perf - WARNING - full garbage collections took 48% CPU time recently (threshold: 10%)
2024-10-12 19:14:07,413 - distributed.utils_perf - WARNING - full garbage collections took 47% CPU time recently (threshold: 10%)
2024-10-12 19:14:10,646 - distributed.utils_perf - WARNING - full garbage collections took 42% CPU time recently (threshold: 10%)
2024-10-12 19:14:15,259 - distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)


In [19]:
def replace_special_chars(content):
    if(content == None or content == ''):
        return ''
    return content.replace('\n',' ').replace('\t',' ')

In [20]:
links_df = dd.read_sql_table('links',conn,'id')


In [21]:
ps_df = df2.compute().drop(columns=['content']).merge(links_df.compute(),how='left',left_on='url',right_on='source')


2024-10-12 19:14:27,592 - distributed.utils_perf - WARNING - full garbage collections took 33% CPU time recently (threshold: 10%)
2024-10-12 19:14:28,462 - distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)
2024-10-12 19:14:53,666 - distributed.utils_perf - WARNING - full garbage collections took 44% CPU time recently (threshold: 10%)
2024-10-12 19:14:54,110 - distributed.utils_perf - WARNING - full garbage collections took 44% CPU time recently (threshold: 10%)
2024-10-12 19:14:54,286 - distributed.utils_perf - WARNING - full garbage collections took 44% CPU time recently (threshold: 10%)
2024-10-12 19:14:54,486 - distributed.utils_perf - WARNING - full garbage collections took 44% CPU time recently (threshold: 10%)
2024-10-12 19:14:54,687 - distributed.utils_perf - WARNING - full garbage collections took 44% CPU time recently (threshold: 10%)


In [22]:
count_df = album_df.compute().drop(columns=['content']).merge(ps_df,how='left',left_on='url',right_on='destination')    
join_df=count_df.groupby('destination')['source'].count().reset_index()

2024-10-12 19:14:58,852 - distributed.utils_perf - WARNING - full garbage collections took 32% CPU time recently (threshold: 10%)
2024-10-12 19:15:03,598 - distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
2024-10-12 19:15:11,582 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)


In [23]:
album_df = album_df.merge(join_df,left_on='url',right_on='destination',how='inner').compute().sort_values(by='source',ascending=False).rename(columns={'source':'rank'})

In [24]:
df3 = album_df[album_df['album_artist']!='<NA>']

In [25]:
vectorizer = TfidfVectorizer(max_features=2000,stop_words='english',min_df =.05,max_df=.95)
meta_df = pd.Series(dtype=str)
vectors = vectorizer.fit_transform(df3['content'].map(replace_special_chars)) # might have to add .compute() after ['content'] in case dask load is lazy

In [26]:
tsne = TSNE(n_components=2, random_state=42)
reducer = umap.UMAP()
list_dense = vectors.todense().tolist()
tnse_df = pd.DataFrame(list_dense,columns = vectorizer.get_feature_names_out())
X_tsne = tsne.fit_transform(tnse_df)
X_umap = reducer.fit_transform(tnse_df)
print(tsne.kl_divergence_)


0.7722000479698181


In [27]:
N_CLUSTERS = 5

In [28]:
kmeans = KMeans(n_clusters=N_CLUSTERS,random_state=42)
kmeans.fit(tnse_df)

KMeans(n_clusters=5, random_state=42)

In [29]:
mask = df3['rank'] > 1 # df3['rank'].mean() 
mask.sum()

1900

In [30]:
hover_names =df3['label'][mask] #.compute()
size = df3['rank'][mask]/df3['rank'].max()
L = len(df3[mask])

In [36]:
fig = px.scatter(x=X_tsne[mask, 0], y=-X_tsne[mask, 1],hover_name=hover_names,color=kmeans.labels_[mask], size=size)
fig.update_layout(
    title=f"t-SNE reduction of {L} TF-IDF measured Wikipedia articles on Jazz albums, weighted by number of citations",

)
fig.update(layout_coloraxis_showscale=False)
py.plot(fig,filename=f'jazz-albums-{L}',auto_open=True)
fig.show()


In [32]:
df3[df3['title'].str.find('Kind of Blue') >= 0].groupby('label').count().reset_index()

,label,url,content,added,updated,pagetype,title,album_artist,destination,rank
0,Kind of Blue by Miles Davis,2,2,2,1,2,2,2,2,2
